In [8]:
import sys 
import pandas as pd
import spacy
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, classification_report, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingRegressor
from tqdm import tqdm
from sklearn.tree import DecisionTreeRegressor
import random
import pickle

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers



In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def custom_mean_squared_error(y_hat, y):
    return np.sum((y_hat - y)**2 * ((abs(3.97 - y)/(0.417342))**2 + 1))/y.shape[0]

In [11]:
def compare(word):
    if word in words['NOUN']:
        return words['NOUN'][word]
    elif word in words['VERB']:
        return words['VERB'][word]
    elif word in words['ADJ']:
        return words['ADJ'][word]
    else:
        return 0

In [12]:
def read(filename):
    with open('./drive/My Drive/ML Project/' + filename + '.json', 'r') as file:
        data = json.load(file)
    return data

In [13]:
summary = read('summary_train')
dictionary = summary['dictionary']
dictionary = list(set(dictionary))
words = summary['words']
dictionary = sorted(dictionary, key=compare, reverse=True)

In [14]:
word_index = {}

for i in range(len(dictionary)):
    word_index[dictionary[i]] = i

summary = read('summary_test')

summary_test = []
data_test = pd.read_csv('./drive/My Drive/ML Project/data/data_test.csv', na_values='.')

for i in tqdm(range(len(summary['X']))):
    new_x = [0 for i in range(len(dictionary))]
    if(summary['imageLink'][i]!=data_test['imageLink'][i]):
      print("ouch")
    for word in summary['X'][i]:
        if word not in word_index:
            continue
        new_x[word_index[word]] += 1
    summary_test.append(new_x)

numerical_test = (data_test.values[:, [4, 5, 9, 11, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]]).astype('float64')

100%|██████████| 4052/4052 [00:01<00:00, 2096.48it/s]


In [15]:
def model(input_shape, regularization = 0):
    X_input = Input(input_shape)
    X = Dense(50, activation='tanh',kernel_regularizer=regularizers.l2(regularization))(X_input)
    X = Dense(50, activation='relu',kernel_regularizer=regularizers.l2(regularization))(X)
    X = Dense(50, activation='relu',kernel_regularizer=regularizers.l2(regularization))(X)
    X = Dense(1)(X)

    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model

In [19]:
X = np.array(summary_test)
happyModel = model((X.shape[1]))
happyModel.compile(optimizer = 'adam', loss = "mse", metrics = ["mse"])
weights = pickle.load(open('./drive/My Drive/ML Project/weights/EnsembleSummary.sav', 'rb'))
happyModel.set_weights(weights)

clf = pickle.load(open('./drive/My Drive/ML Project/weights/EnsembleRF.sav', 'rb'))

ensemble_model = pickle.load(open('./drive/My Drive/ML Project/weights/ensemble_model.sav', 'rb'))

X = np.array(summary_test)
Y = data_test.values[:,6]
X = np.asarray(X).astype('float64')
Y = np.asarray(Y).astype('float64')
X_testNN = happyModel.predict(x = X)

X = np.array(numerical_test)
Y = data_test.values[:,6]
X[:, 3] = np.log(X[:, 3] + 1)
X_testRF = clf.predict(X)

finalTest = []
for i in range(len(X_testRF)):
  finalTest.append([X_testNN[i], X_testRF[i]])

y_hat = ensemble_model.predict(finalTest)
print(mean_squared_error(Y, y_hat))
print(custom_mean_squared_error(y_hat, Y))

0.08461166261608331
0.3927421691700572


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
